In [1]:
import commonutils
import warnings

warnings.simplefilter("ignore")

supersetnames = {"BARRIER_HEIGHTS" : \
                       ["BH76","BHDIV10","BHPERI",\
                        "BHROT27","INV24","PX13","WCPT18"] \
                    ,"INTRAMOLECULAR_INTERACTIONS" : \
                       ["ACONF",'Amino20x4',"BUT14DIOL",\
                        "ICONF","IDISP","MCONF",\
                        "PCONF21","SCONF","UPU23"] , \
                    "SMALL_MOLECULES" :\
                        ["AL2X6","ALK8","ALKBDE10","BH76RC",\
                         "DC13","DIPCS10","FH51","G21EA",\
                         "G21IP","G2RC","HEAVYSB11","NBPRC",\
                         "PA26","RC21","SIE4x4","TAUT15",\
                         "W4-11","YBDE18"], \
                    "INTERMOLECULAR_INTERACTIONS" :\
                       ["ADIM6","AHB21","CARBHB12",\
                        "CHB6","HAL59","HEAVY28","IL16",\
                        "PNICO23","RG18","S22","S66","WATER27"] , \
                    "LARGE_SYSTEMS" :\
                        ["BSR36","C60ISO","CDIE20","DARC",\
                         "ISO34","ISOL24","MB16-43","PArel",\
                            "RSE43"]}  

howmanydifs = 3
methods = {}
entries = ["Nuclear Repulsion  :", \
        "One Electron Energy:", \
        "Two Electron Energy:", \
        "Potential Energy   :", \
        "Kinetic Energy     :", \
        "E(X)               :"  , \
        "E(C)               :"  , \
        "Dispersion correction", \
        "FINAL SINGLE POINT ENERGY"]

functionals = ["PBE"]
basis_sets = ["MINIX", "SVP","TZVP"]
for func in functionals:
    for basis in basis_sets:
        methods[func + "-" + basis] = []
        for ebtry in entries:
            methods[func + "-" + basis].append(ebtry)

# read all the data and initialize the data structures
rootdir = "../Datasets/FullDataSet/"   
allvalues_perset, fullsetnames, models_results = \
        commonutils.read_and_init (rootdir, supersetnames, howmanydifs, methods, \
                       DEBUG=True)

Reading dataset:  BH76
Number of samples:  76
Number of basic descriptors:  76

Reading dataset:  BHDIV10
Number of samples:  10
Number of basic descriptors:  10

Reading dataset:  BHPERI
Number of samples:  26
Number of basic descriptors:  26

Reading dataset:  BHROT27
Number of samples:  27
Number of basic descriptors:  27

Reading dataset:  INV24
Number of samples:  24
Number of basic descriptors:  24

Reading dataset:  PX13
Number of samples:  13
Number of basic descriptors:  13

Reading dataset:  WCPT18
Number of samples:  18
Number of basic descriptors:  18

Reading dataset:  ACONF
Number of samples:  15
Number of basic descriptors:  15

Reading dataset:  Amino20x4
Number of samples:  80
Number of basic descriptors:  80

Reading dataset:  BUT14DIOL
Number of samples:  64
Number of basic descriptors:  64

Reading dataset:  ICONF
Number of samples:  17
Number of basic descriptors:  17

Reading dataset:  IDISP
Number of samples:  6
Number of basic descriptors:  6

Reading dataset:  

In [2]:
# write all the data to a file
#import json
#with open('allvalues_perset.json', 'w') as fp:
#    json.dump(allvalues_perset, fp)

import pickle
pickle.dump(allvalues_perset, open("allvalues_perset.p", "wb"))
pickle.dump(fullsetnames, open("fullsetnames.p", "wb"))
pickle.dump(methods, open("methods.p", "wb"))
pickle.dump(functionals, open("functionals.p", "wb"))
pickle.dump(basis_sets, open("basis_sets.p", "wb"))
pickle.dump(supersetnames, open("supersetnames.p", "wb"))